In [ ]:
!pip install scikit-learn==0.24.2

## CrossValidation Method
- In TimeSeries, TimeSeriesSplit usually is used for cross-validation. However, this will limit the number of training data used for training.
- According to "Mörke, Mathis. "Marcos López de Prado: Advances in financial machine learning." (2019): 491-493.", we can also use PurgedKFold to do the cross-validation. In this schema, the purpose is to find the model performance under different market temperature. So we need to mannually find a split of market and test the model performance. Also, in order to prevent look-ahead bias, we use one month gap before and after the test data size.

In [ ]:
TEST_DAY = 3 * 30
# train_day = 6 * 30
TRAIN_DAY = -1
GAP_DAY = 15
N_SPLIT = 5
CKPT = "ckpt"
# SKIPS = ['Maker', "Monero", "Stellar"]
SKIPS = []

MODEL_PARAMS = {
    "n_estimators": 1000,
    "early_stopping_round": 50,
    "max_depth": 4,  # choose a very shallow depth to ovoid overfitting.
    "random_seed": 2021,
    "learning_rate": 1e-3,
    "colsample_bytree": 0.3,  # For the most of the time, trader only looks at <= 5 features to make decision. Accordingly, we limite the feature-wise sample size.
    "subsample": 0.8,
    "metric": "custom",
    "verbosity": -1,
    "min_data_in_leaf": 100,
    "device": "gpu"
}

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb 
import sklearn
import os
import json
from scipy.stats import pearsonr
import logging

def pearson_eval(preds, train_data):
    """customized lgb evaluation method """
    labels = np.nan_to_num(train_data.get_label())
    return 'corr', pearsonr(labels, np.nan_to_num(preds))[0], True

logger = logging.getLogger()
logger.setLevel(logging.INFO)
lgb.register_logger(logger)
def weighted_correlation(a, b, weights):
    w = np.ravel(weights)
    a = np.ravel(a)
    b = np.ravel(b)

    sum_w = np.sum(w)
    mean_a = np.sum(a * w) / sum_w
    mean_b = np.sum(b * w) / sum_w
    var_a = np.sum(w * np.square(a - mean_a)) / sum_w
    var_b = np.sum(w * np.square(b - mean_b)) / sum_w

    cov = np.sum((a * b * w)) / np.sum(w) - mean_a * mean_b
    corr = cov / (np.sqrt(var_a * var_b) + 1e-12)
    return corr

def validate_one_symble(model, features, label):
    pred = model.predict(features)
    dummy_weights = np.ones_like(pred)
    corr = weighted_correlation(label, pred, dummy_weights)
    return corr

def neutralize_series(series : pd.Series, by : pd.Series, proportion=1.0):
    """
    neutralize pandas series (originally from the Numerai Tournament)
    """
    scores = np.nan_to_num(series.values).reshape(-1, 1)
    exposures = np.nan_to_num(by.values).reshape(-1, 1)
    exposures = np.hstack((exposures, np.array([np.mean(np.nan_to_num(series.values))] * len(exposures)).reshape(-1, 1)))
    correction = proportion * (exposures.dot(np.linalg.lstsq(exposures, scores)[0]))
    corrected_scores = scores - correction
    neutralized = pd.Series(corrected_scores.ravel(), index=series.index)
    return neutralized

def feature_exposures(df, prediction_name = 'Target'):
    feature_names = features
    exposures = []
    for f in feature_names:
        fe = np.corrcoef(np.nan_to_num(df[prediction_name].values), np.nan_to_num(df[f].values))[0, 1]
        exposures.append(fe)
    return np.array(exposures)

def max_feature_exposure(df): return np.max(np.abs(feature_exposures(df)))
def feature_exposure(df): return np.sqrt(np.mean(np.square(feature_exposures(df))))

In [ ]:
# Two new features from the competition tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

# A utility function to build features from the original df
# It works for rows to, so we can reutilize it.
def get_features(df, row=False):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['Upper_Shadow'] = upper_shadow(df_feat)
    df_feat['Lower_Shadow'] = lower_shadow(df_feat)
    
    
    df_feat["Close/Open"] = df_feat["Close"] / df_feat["Open"] 
    df_feat["Close-Open"] = df_feat["Close"] - df_feat["Open"] 
    df_feat["High-Low"] = df_feat["High"] - df_feat["Low"] 
    df_feat["High/Low"] = df_feat["High"] / df_feat["Low"]
    if row:
        df_feat['Mean'] = df_feat[['Open', 'High', 'Low', 'Close']].mean()
    else:
        df_feat['Mean'] = df_feat[['Open', 'High', 'Low', 'Close']].mean(axis=1)
    
    df_feat['High/Mean'] = df_feat['High'] / df_feat['Mean']
    df_feat['Low/Mean'] = df_feat['Low'] / df_feat['Mean']
    df_feat['Volume/Count'] = df_feat['Volume'] / (df_feat['Count'] + 1)

    ## possible seasonality, datetime  features (unlikely to me meaningful, given very short time-frames)
    ### to do: add cyclical features for seasonality
    times = pd.to_datetime(df["timestamp"],unit="s",infer_datetime_format=True)
    if row:
        df_feat["hour"] = times.hour  # .dt
        df_feat["dayofweek"] = times.dayofweek 
        df_feat["day"] = times.day 
    else:
        df_feat["hour"] = times.dt.hour  # .dt
        df_feat["dayofweek"] = times.dt.dayofweek 
        df_feat["day"] = times.dt.day 
    #df_feat.drop(columns=["time"],errors="ignore",inplace=True)  # keep original epoch time, drop string

    return df_feat


def get_Xy_and_model_for_asset(df_train, asset_id):
    df = df_train[df_train["Asset_ID"] == asset_id]
    
    # TODO: Try different features here!
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.replace([np.inf, -np.inf], np.nan).dropna(how="any")
    
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]

    # TODO: Try different models here!
    model = LGBMRegressor(n_estimators=10)
    model.fit(X, y)
    return X, y, model

In [ ]:
df = pd.read_feather("../input/filledtraindata/train.feather")
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
asset_df = pd.read_csv("../input/g-research-crypto-forecasting/asset_details.csv", index_col="Asset_Name")

In [ ]:
weights = asset_df["Weight"]
weights = weights / weights.sum()

In [ ]:
close_df = df.pivot(index="datetime", columns="Asset_ID", values="Close")
close_df = close_df.fillna(method="ffill")
close_df = close_df.pct_change(1)
weighted_ret = close_df.multiply(other=weights).sum(axis=1).cumsum()
weighted_ret.plot()

In [ ]:
period1 = slice("2018-1-1", "2018-11-1")  # 178560
weighted_ret.loc[period1].plot()

In [ ]:
period2 = slice("2019-1-1", "2019-7-1")
weighted_ret.loc[period2].plot()

In [ ]:
period3 = slice("2019-8-1", "2019-12-1")
weighted_ret.loc[period3].plot()

In [ ]:
period4 = slice("2020-1-1", "2021-5-1")
weighted_ret.loc[period4].plot()

In [ ]:
period5 = slice("2021-6-1", "2021-10-1")
weighted_ret.loc[period5].plot()

In [ ]:
TEST_SCORE_DF = pd.DataFrame(index=asset_df.index, columns=[period1.start, period2.start, period3.start, period4.start, period5.start])
TRAIN_SCORE_DF = pd.DataFrame(index=asset_df.index, columns=[period1.start, period2.start, period3.start, period4.start, period5.start])

In [ ]:
def get_score_for_one_symbol_new_cv(all_df, asset_id, dry_run=False, model_params={}, dump_root="ckpt"):
    symbol_df = all_df[all_df.Asset_ID == asset_id].fillna(method="ffill").dropna()
    symbol_df = symbol_df.set_index("datetime")
    if asset_id == "Maker":
        symbol_df = symbol_df.loc["2020-08-04":]
    elif asset_id == "Monero":
        symbol_df = symbol_df.loc["2018-11-05":]
    elif asset_id == "Stellar":
        symbol_df = symbol_df.loc["2018-07-14":]
    train_score_by_cv = [0] * N_SPLIT
    test_score_by_cv = [0] * N_SPLIT
    train_size_by_cv = [0] * N_SPLIT
    test_size_by_cv = [0] * N_SPLIT
    test_period_by_cv = [0] * N_SPLIT
    test_type_by_cv = [0] * N_SPLIT
    iter_by_cv = [0] * N_SPLIT
    df_proc = get_features(symbol_df)
    bulls = [period4, period2]
    bears = [period3]
    neutral = [period1, period5]
    for i, period in enumerate([period1, period2, period3, period4, period5]):
        print(period)
        train_features, train_target = df_proc.loc[period], symbol_df["Target"].loc[period]
        test_features, test_target = df_proc.loc[period], symbol_df["Target"].loc[period]
        if test_features.size == 0:
            continue
        part1 = pd.Timestamp(period.start) - pd.Timedelta("1m")
        part2 = pd.Timestamp(period.stop) + pd.Timedelta("1m")
        dfs = []
        targets = []
        _df1 = df_proc.loc[:part1]
        _df2 = df_proc.loc[part2:]
        if _df1.size > 0:
            dfs.append(_df1)
            targets.append(symbol_df["Target"].loc[:part1])
        if _df2.size > 0:
            dfs.append(_df2)
            targets.append(symbol_df["Target"].loc[part2:])
        if len(dfs) == 2:
            train_features = pd.concat(dfs)
            train_target = pd.concat(targets)
        elif len(dfs) == 1:
            train_features = dfs[0]
            train_target = targets[0]
        else:
            continue
        train_size = len(train_features)
        test_size = len(test_features)
        train_features = train_features.replace([np.inf, -np.inf], np.nan).dropna(how="any")
        test_features = test_features.replace([np.inf, -np.inf], np.nan).dropna(how="any")
        train_set = lgb.Dataset(train_features, label=train_target)
        test_set = lgb.Dataset(test_features, label=test_target)
        # continuous
        assert len(train_features) == len(train_target)
        assert len(test_features) == len(test_target)
        booster = lgb.train(train_set=train_set, params=model_params, valid_sets=[test_set], feval=pearson_eval)
        corr_train = validate_one_symble(booster, train_features, train_target)
        corr_test = validate_one_symble(booster, test_features, test_target)
#         print("Score on Train[{}]: {:.4f}".format(i, corr_train))
#         print("Score on Test[{}]: {:.4f}".format(i, corr_test))
        TEST_SCORE_DF.loc[asset_id, period.start] = float(corr_test)
        TRAIN_SCORE_DF.loc[asset_id, period.start] = float(corr_train)
        train_score_by_cv[i] = float(corr_train)
        test_score_by_cv[i] = float(corr_test)
        train_size_by_cv[i] = int(train_size)
        test_size_by_cv[i] = int(test_size)
        test_period_by_cv[i] = [period.start, period.stop]
        if period in bulls:
            test_type_by_cv[i] = "bull"
        elif period in bears:
            test_type_by_cv[i] = "bear"
        else:
            test_type_by_cv[i] = "neutral"
        iter_by_cv[i] = booster.best_iteration
        str_path = os.path.join(os.getcwd(), dump_root, asset_id, str(i))
        os.makedirs(str_path, exist_ok=True)
        model_str = booster.model_to_string()
        with open(os.path.join(str_path, "lgb.ckpt"), "w") as f:
            f.write(model_str)
        
        if dry_run:
            break
    avg_train_score = sum(train_score_by_cv) / N_SPLIT
    avg_test_score = sum(test_score_by_cv) / N_SPLIT
    best_iteration = booster.best_iteration
    meta = {
            "train_score": train_score_by_cv,
            "test_score": test_score_by_cv,
            "train_size_by_cv": train_size_by_cv,
            "test_size_by_cv": test_size_by_cv,
            "test_type_by_cv": test_type_by_cv,
            "test_period_by_cv": test_period_by_cv,
            "model_params": model_params,
            "avg_train_score": avg_train_score,
            "avg_test_score": avg_test_score,
            "iter_by_cv": iter_by_cv
        }
        
    meta_path = os.path.join(os.getcwd(), dump_root, asset_id, "lgb_meta.json")
    with open(meta_path, "w") as f:
        f.write(json.dumps(meta, indent=2))
    return avg_train_score, avg_test_score, meta

In [ ]:
train_score_by_symbol = {}
test_score_by_symbol = {}

for asset_id in df.Asset_ID.unique():
#     if asset_id in SKIPS:
#         print("Skip ", asset_id)
#         continue
    print(asset_id + "\n***")
#     train_score, test_score, meta = get_score_for_one_symbol(df, asset_id, dry_run=False, model_params=MODEL_PARAMS, dump_root=CKPT)
    train_score, test_score, meta = get_score_for_one_symbol_new_cv(df, asset_id, dry_run=False, model_params=MODEL_PARAMS, dump_root=CKPT)
    train_score_by_symbol[asset_id] = train_score
    test_score_by_symbol[asset_id] = test_score
    
    print(meta)
    print("\n")
    

In [ ]:
TRAIN_SCORE_DF

In [ ]:
TEST_SCORE_DF

In [ ]:
TRAIN_SCORE_DF.to_csv(os.path.join(os.getcwd(), CKPT, "train_score_df.csv"))
TEST_SCORE_DF.to_csv(os.path.join(os.getcwd(), CKPT, "test_score_df.csv"))

In [ ]:
final_train_score = sum([score * weights[s] for s, score in train_score_by_symbol.items()])
final_test_score = sum([score * weights[s] for s, score in test_score_by_symbol.items()])
print("avg. model score on train: {:.4f}".format(final_train_score))
print("avg. model score on test: {:.4f}".format(final_test_score))

In [ ]:
score_by_symbol = pd.DataFrame({"train_score": train_score_by_symbol, "test_score": test_score_by_symbol}).sort_values(by="train_score")

In [ ]:
score_by_symbol